In [16]:
%reload_ext autoreload
%autoreload 2

In [3]:
os.chdir('/Users/akhileshvarmabhupathiraju/Documents/taxi_ml_project/NYC_taxi_predictor/')


In [17]:
import src.config as config

In [18]:
from datetime import datetime, timedelta

import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H')
print(f'{current_date=}')

# we fetch raw data for the last 28 days, to add redundancy to our data pipeline
fetch_data_to = current_date
fetch_data_from = current_date - timedelta(days=28)

current_date=Timestamp('2023-06-10 20:00:00')


In [19]:
from src.data import load_raw_data

def fetch_batch_raw_data(from_date: datetime, to_date: datetime) -> pd.DataFrame:
    """
    Simulate production data by sampling historical data from 52 weeks ago (i.e. 1 year)
    """
    from_date_ = from_date - timedelta(days=7*52)
    to_date_ = to_date - timedelta(days=7*52)
    print(f'{from_date=}, {to_date_=}')

    # download 2 files from website
    rides = load_raw_data(year=from_date_.year, months=from_date_.month)
    rides = rides[rides.pickup_datetime >= from_date_]
    rides_2 = load_raw_data(year=to_date_.year, months=to_date_.month)
    rides_2 = rides_2[rides_2.pickup_datetime < to_date_]

    rides = pd.concat([rides, rides_2])

    # shift the data to pretend this is recent data
    rides['pickup_datetime'] += timedelta(days=7*52)

    rides.sort_values(by=['pickup_location_id', 'pickup_datetime'], inplace=True)

    return rides

In [20]:
rides = fetch_batch_raw_data(from_date=fetch_data_from, to_date=fetch_data_to)

from_date=Timestamp('2023-05-13 20:00:00'), to_date_=Timestamp('2022-06-11 20:00:00')
File 2022-05 was already in the local storage
File 2022-06 was already in the local storage


In [21]:
from src.data import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

          pickup_hour  pickup_location_id   0
0 2023-05-13 20:00:00                   4  16
1 2023-05-13 20:00:00                   7   3
2 2023-05-13 20:00:00                  10   4
3 2023-05-13 20:00:00                  13  27
4 2023-05-13 20:00:00                  17   1
Index(['pickup_hour', 'pickup_location_id', 0], dtype='object')
Index(['pickup_hour', 'pickup_location_id', 'rides'], dtype='object')
(68986, 3)


In [22]:
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_or_create_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

AttributeError: module 'src.config' has no attribute 'HOPSWORKS_PROJECT_NAME'

In [ ]:
feature_group.insert(ts_data, write_options={"wait_for_job": True})

NameError: name 'feature_group' is not defined